# logging for warnings

In [ ]:

# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest main of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 26.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-89e2dnw_/farm-haystack_d9734440b3b743b2be61567b26c86a93
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-89e2dnw_/farm-haystack_d9734440b3b743b2be61567b26c86a93
  Resolved https://github.com/deepset-ai/haystack.git to commit b84a6b17165dbf10665794b5decdefa113300749
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [ ]:

import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

#  import the necessary libraries 

In [ ]:

from pprint import pprint
from tqdm import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

In [ ]:
# Option 2: In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

# import sentence mapping 

In [ ]:
import pandas as pd 
df = pd.read_csv('/content/sentence_mapping.csv')
df.head()

,Unnamed: 0,Phrases,Paragraphs
0,0,deep learning,Deep learning architectures are classified in the following parts Artificial...
1,1,machine learning,What is Machine Learning Machine Learning is an application of Artificial In...
2,2,artificial intelligence,What is Machine Learning Machine Learning is an application of Artificial In...
3,3,artificial neural network,Deep learning architectures are classified in the following parts Artificial...
4,4,intelligent beings,The idea of inanimate objects coming to life as intelligent beings have been...


In [ ]:
parg = []
for k  in range(len(df['Paragraphs'])) : 
  parg.append(df['Phrases'][k] + ' ' + df['Paragraphs'][k])
parg   


['deep learning Deep learning architectures are classified in the following parts Artificial Neural NetworkANN Artificial Neural Network is the computing system designed to stimulate the way the human brain analyzes and processes information.',
 'machine learning What is Machine Learning Machine Learning is an application of Artificial Intelligence that enables a system to automatically learn and improve from experience without being explicitly programmed.',
 'artificial intelligence What is Machine Learning Machine Learning is an application of Artificial Intelligence that enables a system to automatically learn and improve from experience without being explicitly programmed.',
 'artificial neural network Deep learning architectures are classified in the following parts Artificial Neural NetworkANN Artificial Neural Network is the computing system designed to stimulate the way the human brain analyzes and processes information.',
 'intelligent beings The idea of inanimate objects comi

In [ ]:
docs = []
for i in parg:
  docs.append({"content": i})
docs  

[{'content': 'deep learning Deep learning architectures are classified in the following parts Artificial Neural NetworkANN Artificial Neural Network is the computing system designed to stimulate the way the human brain analyzes and processes information.'},
 {'content': 'machine learning What is Machine Learning Machine Learning is an application of Artificial Intelligence that enables a system to automatically learn and improve from experience without being explicitly programmed.'},
 {'content': 'artificial intelligence What is Machine Learning Machine Learning is an application of Artificial Intelligence that enables a system to automatically learn and improve from experience without being explicitly programmed.'},
 {'content': 'artificial neural network Deep learning architectures are classified in the following parts Artificial Neural NetworkANN Artificial Neural Network is the computing system designed to stimulate the way the human brain analyzes and processes information.'},
 {'

# Elastic search 

In [ ]:
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)
question_generator = QuestionGenerator()

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Using sep_token, but it is not set yet.


# Question generator by using sentence mapping 

In [ ]:
question_generation_pipeline = QuestionGenerationPipeline(question_generator)
for idx, document in enumerate(document_store):
  print(f"\n * Generating questions for document {idx}: {document.content[:100]}...\n")
  result = question_generation_pipeline.run(documents=[document])
  print_questions(result)


 * Generating questions for document 0: deep learning Deep learning architectures are classified in the following parts Artificial Neural Ne...


Generated questions:
 - Deep learning architectures are classified in what parts?
 - Artificial Neural Network is the computing system designed to stimulate the way the human brain analyzes and processes information?

 * Generating questions for document 1: machine learning What is Machine Learning Machine Learning is an application of Artificial Intellige...


Generated questions:
 - What is machine learning?
 - Machine learning is an application of Artificial Intelligence that enables a system to automatically learn and improve from what?

 * Generating questions for document 2: artificial intelligence What is Machine Learning Machine Learning is an application of Artificial In...


Generated questions:
 - What is Machine Learning?
 - Machine Learning enables a system to learn and improve from what?

 * Generating questions for document 3:

# Question/Answer generator

In [ ]:
reader = FARMReader("deepset/roberta-base-squad2")
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
for idx, document in enumerate(tqdm(document_store)):
  print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
  result = qag_pipeline.run(documents=[document])
  print_questions(result)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
0it [00:00, ?it/s]


 * Generating questions and answers for document 0: deep learning Deep learning architectures are classified in the following parts Artificial Neural Ne...



...


Generated pairs:
 - Q: What can algorithms be classified into?
      A: Supervised Learning
 - Q: What is Supervised Learning?
      A: Imagine you are learning a task under supervision

 * Generating questions and answers for document 22: human brain Deep learning architectures are classified in the following parts Artificial Neural Netw...




Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 31.99 Batches/s]
23it [00:12,  2.06it/s]


Generated pairs:
 - Q: What are deep learning architectures classified into?
      A: Artificial Neural Network
 - Q: What is the computing system designed to stimulate?
      A: the way the human brain analyzes and processes information

 * Generating questions and answers for document 23: field But the field of AI was not formally founded until at a conference at Dartmouth College, in Ha...




Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 31.93 Batches/s]
24it [00:13,  1.74it/s]


Generated pairs:
 - Q: When was the field of AI founded?
      A: until at a conference at Dartmouth College
 - Q: What was the name of the conference held at Dartmouth College in Hanover, New Hampshire?
      A: AI
